### Preparation

In [3]:
prepare = False
if prepare == True:
    !pip install pytextrank
    !python -m spacy download pl_core_news_sm #or lg

In [4]:
import pandas as pd
import numpy as np
import pytextrank
import spacy
from icecream import ic

In [5]:
text_pl = "Wikipedia – wielojęzyczna encyklopedia internetowa działająca zgodnie z zasadą otwartej treści. Funkcjonuje w oparciu o oprogramowanie MediaWiki (haw. wiki – „szybko”, „prędko”), wywodzące się z koncepcji WikiWikiWeb, umożliwiające edycję każdemu użytkownikowi odwiedzającemu stronę i aktualizację jej treści w czasie rzeczywistym. Słowo Wikipedia jest neologizmem powstałym w wyniku połączenia wyrazów wiki i encyklopedia. Slogan Wikipedii brzi: „Wolna encyklopedia, którą każdy może redagować”. Serwis był notowany w rankingu Alexa na miejscu 13[1]. "
text_en = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."

## Pytextrank
https://pypi.org/project/pytextrank/

### English pytextrank

In [19]:
# English version from textrank example
if prepare == True:
    !python -m spacy download en_core_web_sm
    
# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
nlp.pipe_names



['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner',
 'textrank']

In [20]:
doc = nlp(text_en)

pytextrank_result_en = doc._.phrases

# examine top-ranked phrases in the document
for phrase in pytextrank_result_en[:5]:
    ic(phrase)

tr = doc._.textrank

if prepare == True:
    !pip install "altair"
    !pip install 'pytextrank[viz]'
tr.plot_keyphrases()

ic| phrase: Phrase(text='mixed types', chunks=[mixed types], count=1, rank=0.18359439311764025)
ic| phrase: Phrase(text='systems', chunks=[systems, systems, systems], count=3, rank=0.1784796193107821)
ic| phrase: Phrase(text='minimal generating sets', chunks=[minimal generating sets], count=1, rank=0.15037838042245094)
ic| phrase: Phrase(text='nonstrict inequations', chunks=[nonstrict inequations], count=1, rank=0.14740065982407313)
ic| phrase: Phrase(text='strict inequations', chunks=[strict inequations], count=1, rank=0.13946027725597837)


alt.Chart(...)

In [21]:
for chunk in doc.noun_chunks:
    ic(chunk)

ic| chunk: Compatibility
ic| chunk: systems
ic| chunk: linear constraints
ic| chunk: the set
ic| chunk: natural numbers
ic| chunk: Criteria
ic| chunk: compatibility
ic| chunk: a system
ic| chunk: linear Diophantine equations
ic| chunk: strict inequations
ic| chunk: nonstrict inequations
ic| chunk: Upper bounds
ic| chunk: components
ic| chunk: a minimal set
ic| chunk: solutions
ic| chunk: algorithms
ic| chunk: construction
ic| chunk: minimal generating sets
ic| chunk: solutions
ic| chunk: all types
ic| chunk: systems
ic| chunk: These criteria
ic| chunk: the corresponding algorithms
ic| chunk: a minimal supporting set
ic| chunk: solutions
ic| chunk: all the considered types systems
ic| chunk: systems
ic| chunk: mixed types


### Polski pytextrank

In [21]:
from spacy.matcher import Matcher
from spacy.attrs import POS
from spacy.tokens import Doc

def get_chunks(doc):
    ## For slovak language
    #TODO: change to polish
    np_label = doc.vocab.strings.add("NP")
    nlp = spacy.load('pl_core_news_sm') 
    matcher = Matcher(nlp.vocab)
    pattern = [{POS: 'ADJ', "OP": "+"}, {POS: {"IN": ["NOUN", "PROPN"]}, "OP": "+"}]
    matcher.add("Adjective(s), (p)noun", [pattern])
    matches = matcher(doc)

    for match_id, start, end in matches:
        yield start, end, np_label      

In [22]:
# List the tokens including their lemmas and POS tags
# spacy_udpipe.download("pl") # download model
spacy.lang.pl.PolishDefaults.syntax_iterators = {"noun_chunks" : get_chunks}  #noun_chunk replacement

nlp_pl = spacy.load('pl_core_news_sm') # or lg
print(nlp_pl)
# nlp_pl = spacy_udpipe.load("pl")

nlp_pl.add_pipe("textrank")
nlp_pl.pipe_names

['tok2vec',
 'morphologizer',
 'parser',
 'lemmatizer',
 'tagger',
 'attribute_ruler',
 'ner',
 'textrank']

In [23]:
doc = nlp_pl(text_pl)

doc._.phrases 

[Phrase(text='otwartej treści', chunks=[otwartej treści], count=1, rank=0.15272256587280342),
 Phrase(text='odwiedzającemu stronę', chunks=[odwiedzającemu stronę], count=1, rank=0.12311659088575572),
 Phrase(text='umożliwiające edycję', chunks=[umożliwiające edycję], count=1, rank=0.11637146094468781),
 Phrase(text='Wolna encyklopedia', chunks=[Wolna encyklopedia], count=1, rank=0.1077646940828873),
 Phrase(text='wielojęzyczna encyklopedia', chunks=[wielojęzyczna encyklopedia], count=1, rank=0.10263387240609723),
 Phrase(text='Alexa', chunks=[Alexa], count=1, rank=0.0643230111160754),
 Phrase(text='Wikipedia', chunks=[Wikipedia, Wikipedia], count=2, rank=0.052553914024320754),
 Phrase(text='Wikipedii', chunks=[Wikipedii], count=1, rank=0.052553914024320754)]

In [24]:
for token in doc:
    print(token.text, token.lemma_, token.tag_, token.pos_, token.sentiment)
    break

Wikipedia Wikipedia SUBST PROPN 0.0


## Summa
https://pypi.org/project/summa/

In [ ]:
if prepare == True:
    !pip install summa

from summa import keywords

### English summma – comparision with pytextrank

In [ ]:
summa_result_en = keywords.keywords(text_en, words=5, split=True, scores=True)

pytextrank_result_en_list = [(phrase.text, phrase.rank) for phrase in pytextrank_result_en] 

In [ ]:
summa_result_en

[('types', 0.3532248258785909),
 ('numbers', 0.29945212418790307),
 ('upper', 0.24567942249721883),
 ('minimal', 0.24228720922768252),
 ('set', 0.24228720922768227),
 ('sets', 0.24228720922768227)]

In [ ]:
pytextrank_result_en_list[:10]

[('mixed types', 0.18359439311764025),
 ('systems', 0.1784796193107821),
 ('minimal generating sets', 0.15037838042245094),
 ('nonstrict inequations', 0.14740065982407313),
 ('strict inequations', 0.13946027725597837),
 ('linear Diophantine equations', 0.1195023546245721),
 ('natural numbers', 0.11450088293222845),
 ('solutions', 0.10780718173686318),
 ('linear constraints', 0.10529828014583348),
 ('all the considered types systems', 0.1036960590708142)]

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(text_en)

lemmatized_text_en = " ".join([token.lemma_ for token in doc])
summa_result_en = keywords.keywords(lemmatized_text_en, words=15, split=True, scores=True)
summa_result_en

[('diophantine equation', 0.4260698809690522),
 ('strict inequation', 0.42606988096904974),
 ('linear constraint', 0.31955241072679164),
 ('nonstrict', 0.2130349404845244),
 ('consider', 0.016514525023541175),
 ('type', 0.016514525023541077),
 ('corresponding', 0.014030102458728334),
 ('algorithm', 0.014030102458728322),
 ('upper bound', 0.012385893767655882),
 ('number', 0.011726607692410565),
 ('mixed', 0.008257262511770534)]

### Polski – summa

In [ ]:
nlp = spacy.load('pl_core_news_sm') # or lg
doc = nlp(text_pl)

lemmatized_text_pl = " ".join([token.lemma_ for token in doc])
print(text_pl, lemmatized_text_pl, sep = "\n\n")


Wikipedia – wielojęzyczna encyklopedia internetowa działająca zgodnie z zasadą otwartej treści. Funkcjonuje w oparciu o oprogramowanie MediaWiki (haw. wiki – „szybko”, „prędko”), wywodzące się z koncepcji WikiWikiWeb, umożliwiające edycję każdemu użytkownikowi odwiedzającemu stronę i aktualizację jej treści w czasie rzeczywistym. Słowo Wikipedia jest neologizmem powstałym w wyniku połączenia wyrazów wiki i encyklopedia. Slogan Wikipedii brzi: „Wolna encyklopedia, którą każdy może redagować”. Serwis był notowany w rankingu Alexa na miejscu 13[1]. 

Wikipedia – wielojęzyczny encyklopedia internetowy działać zgodnie z zasada otwarty treść . Funkcjonuje w oparcie o oprogramować mediaWiki ( haw . wik – „ szybko ” , „ prędko ” ) , wywodzące się z koncepcja WikiWikiWeb , umożliwiać edycja każdy użytkownik odwiedzający strona i aktualizacja on treść w czas rzeczywisty . słowo Wikipedia być neologizm powstały w wynik połączć wyraz wik i encyklopedia . slogan Wikipedia brzi : „ wolny encyklopedi

In [ ]:
keywords.keywords(lemmatized_text_pl, language="polish", words=10).split("\n")

['wikipedia',
 'encyklopedia',
 'wik',
 'redagować',
 'odwiedzający',
 'otwarty treść',
 'wikiwikiweb',
 'umożliwiać',
 'mediawiki']

In [ ]:
keywords.keywords(text_pl, language="polish", words=10).split("\n")

['encyklopedia',
 'wiki',
 'treści',
 'otwartej',
 'odwiedzającemu',
 'wikiwikiweb',
 'rzeczywistym',
 'prędko',
 'połączenia',
 'mediawiki']